In [ ]:
import numpy as np
import tensorflow as tf
import time

In [ ]:
# GLOBAL VARIABLES
NUM_LABELS = 200  # The number of labels. 40
BATCH_SIZE = 5000  # The number of training examples to use per training step  3000
DM_min,DM_max = 0.05176, 20.0625 # -0.9..20
training_epochs = 1200  #1950 # 40
#display_step = 100

In [ ]:
# Network Parameters
n_hidden_1 = 56 # 1st layer num features # 56
n_hidden_2 = 56 # 2nd layer num features # 56

In [7]:
def ToClass(X, NBIN, XMIN,XMAX):
    labels = (np.array(X-XMIN)/(XMAX-XMIN)*NBIN).astype(int)
    labels[labels < 0] = 0
    labels[labels >= NBIN] = NBIN-1
    return labels

def ToDM(X,NBIN,XMIN,XMAX):
    TEMP1 = (X*(XMAX-XMIN)/NBIN) + XMIN
    return TEMP1

In [ ]:
def extract_data(filename):

    # Arrays to hold the labels and feature vectors.
    labels = []
    fvecs = []

    # Iterate over the rows, splitting the label from the features. Convert labels
    # to integers and features to floats.
    for line in file(filename):
        row = line.split(" ")
        labels.append(int(row[0]))
        fvecs.append([float(x) for x in row[1:]])

    # Convert the array of float arrays into a numpy float matrix.
    fvecs_np = np.matrix(fvecs).astype(np.float32)

    # Convert the array of int labels into a numpy array.
    labels_np = np.array(labels).astype(dtype=np.uint16)
    #DM_np = ToDM(labels_np, NUM_LABELS, DM_min, DM_max)

    # Convert the int numpy array into a one-hot matrix.
    labels_onehot = (np.arange(NUM_LABELS) == labels_np[:, None]).astype(np.float32)

    # Return a pair of the feature matrix and the one-hot label matrix.
    return fvecs_np,labels_onehot

In [ ]:
def extract_data2(filename):
    DM = []
    Cond = []
    
    for line in file(filename):
        row = line.split(" ")
        Cond.append(int(row[0]))
        DM.append(float(row[1]))
        
    DM_np = np.array(DM).astype(dtype=np.float32)
    COND = np.array(Cond).astype(dtype=np.uint16)
    return COND, DM_np
        

In [ ]:
def multilayer_perceptron(_X, _weights, _biases):
    #Hidden layer with RELU activation
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(_X, _weights['h1']), _biases['b1'])) 
    #Hidden layer with RELU activation
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, _weights['h2']), _biases['b2'])) 
    #layer_2_drop = tf.nn.dropout(layer_2, keep_prob)
    return tf.matmul(layer_2, weights['out']) + biases['out']

In [ ]:
# Extract it into numpy arrays.
train_data, train_labels = extract_data('RANDOM_SAMPLE_BIN.txt') # Datos de entrenamiento
test_data, test_labels = extract_data('TESTDATA.txt')            # Datos de test en labels
Condition, test_DM = extract_data2('TEST_DELTAS.txt')            # Datos de test en DarkMatter y condicion True, False

In [ ]:
# Get the shape of the training data.
train_size,num_features = train_data.shape

In [ ]:
# Store layers weight & bias 
weights = {
    'h1': tf.Variable(tf.random_normal([num_features, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, NUM_LABELS]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([NUM_LABELS]))
}

In [ ]:
x = tf.placeholder("float", shape=[None, num_features])
y_ = tf.placeholder("float", shape=[None, NUM_LABELS])

# The output layer.
y = tf.nn.softmax(multilayer_perceptron(x, weights, biases))
    
# Optimization.
cross_entropy = -tf.reduce_sum(y_*tf.log(y))
train_step = tf.train.GradientDescentOptimizer(0.0001).minimize(cross_entropy) # 0.0001
    
# Evaluation.
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
output = tf.argmax(y,1)

In [ ]:
start_time = time.time()

with tf.Session() as s:
        # Run all the initializers to prepare the trainable parameters.
        tf.initialize_all_variables().run()
        # Iterate and train.
        for step in xrange(training_epochs * train_size // BATCH_SIZE):

            offset = (step * BATCH_SIZE) % train_size
            batch_data = train_data[offset:(offset + BATCH_SIZE), :]
            batch_labels = train_labels[offset:(offset + BATCH_SIZE)]
            train_step.run(feed_dict={x: batch_data, y_: batch_labels})
        
            #if step % display_step == 0: 
            #    print "Step:",'%i' % step,"Accuracy:", accuracy.eval(feed_dict={x: test_data, y_: test_labels})
        
        Pred = output.eval(feed_dict={x: test_data})
        print "Test Accuracy:", accuracy.eval(feed_dict={x: test_data, y_: test_labels})
        #for i in range(len(Pred)):
        #   print np.where(test_labels[i])[0][0],Pred[i]

        #print "Test Accuracy:", accuracy.eval(feed_dict={x: test_data, y_: test_labels})
        
print("--- %3fs: loaded ---" % (time.time() - start_time))   
Pred_DM = ToDM(Pred,NUM_LABELS,DM_min,DM_max)
np.savetxt('Exit_01.txt', np.array([Condition, test_DM, Pred_DM]).T, fmt= ['%i','%.5f','%.5f'] )

In [ ]:
for i in range(100):
    print np.where(test_labels[i])[0][0],Pred[i]

In [ ]:
# TRAINING EPOCH = 540   BATCH SIZE = 5000   HIDDEN LAYERS = 28

In [ ]:
Pred_DM = ToDM(Pred,NUM_LABELS,DM_min,DM_max)
np.savetxt('Exit_01.txt', np.array([Condition, test_DM,Pred_DM]).T, fmt= ['%i','%.5f','%.5f'] )
#for i in range(200,300):
#    print np.where(test_labels[i])[0][0],Pred[i]

In [1]:
import math
import matplotlib.pyplot as plt
import matplotlib as mpl
from scipy import stats
import pylab as pl
import pylab
from pylab import *
from matplotlib import rc
import readcol as lector
import numpy as np
rc('font', **{'family': 'serif', 'serif': ['Computer Modern'], 'size': 13})
rc('text', usetex=True)

In [2]:
Cond, delta_DM, delta_ANN = lector.readcol("Exit_01.txt",twod = False) 

In [3]:
Cond = Cond.astype(bool)

In [4]:
ANN_True = delta_ANN[Cond == True]
ANN_False = delta_ANN[Cond == False]
delta_DM_True = delta_DM[Cond == True]
delta_DM_False = delta_DM[Cond == False]

In [12]:
# ====== RESIDUAL PLOT ========
Res_ANN_True = (ANN_True - delta_DM_True) / delta_DM_True
Res_ANN_False = (ANN_False - delta_DM_False) / delta_DM_False
RESIDUO = (delta_ANN - delta_DM) / delta_DM

In [17]:
print len(RESIDUO), len(Res_ANN_False)+ len(Res_ANN_True)

264849 264849


In [27]:
maxval, minval = delta_DM.max(), delta_DM.min()
num_bins = 150
delta = (maxval-minval) / num_bins
i,l  = 0,0

labels_True = ToClass(delta_DM_True, num_bins, minval, maxval) # delta_DM_True
labels_False = ToClass(delta_DM_False, num_bins, minval, maxval) # delta_DM_False
bin_list_True = []
bin_list_False = []

while i < num_bins:
    r = np.any(labels_True == i)
    if r == True:
        bin_list_True.append(i)
    i += 1    
    
while l < num_bins:
    r = np.any(labels_False == l)
    if r == True:
        bin_list_False.append(l)
    l += 1    

bin_list_errobar_T = [k+1 for k in np.arange(0,num_bins,5)]
bin_list_errobar_F = [k+3 for k in np.arange(0,num_bins,5)]    
    

In [25]:
a_0 = minval + delta/2
t = np.arange(maxval)
bins_True = np.array([ a_0 + delta*n for n in bin_list_True])
bins_False = np.array([ a_0 + delta*n for n in bin_list_False])
median_True = np.array([np.median((Res_ANN_True[labels_True == j])) for j in bin_list_True]) # ANN_True
median_False = np.array([np.median((Res_ANN_False[labels_False == j])) for j in bin_list_False]) # ANN_False
#running_err_down_T    = [(np.sort(ANN_True[labels_True == k]))[(len(ANN_True[labels_True == k]))/5] for k in bin_list_errobar_T]
#running_err_up_T      = [(np.sort(ANN_True[labels_True == l]))[(4*len(ANN_True[labels_True == l]))/5] for l in bin_list_errobar_T]
#running_err_down_F    = [(np.sort(ANN_False[labels_False == k]))[(len(ANN_False[labels_False == k]))/5] for k in bin_list_errobar_F]
#running_err_up_F      = [(np.sort(ANN_False[labels_False == l]))[(4*len(ANN_False[labels_False == l]))/5] for l in bin_list_errobar_F]

In [30]:
# ========= RESIDUAL PLOT =========
plt.plot(delta_DM, RESIDUO, 'o', color = '#e0ffff', markersize = .5) #,alpha =.55, markersize = .5
plt.plot(t,t*0,'--', lw = 2.5,color = 'green')
plt.plot(bins_True,median_True,color='red',linestyle = '-',lw=2.0,alpha=.7,label = r"Median $\left(\{\delta_i\}<\delta_c\right)$") 
plt.plot(bins_False,median_False,color='blue',linestyle = '-',lw=2.0,alpha=.7,label = r"Median $\left(\delta_i > \delta_c\right)$") 
plt.xlabel(r"$\delta_{DM} + 1 $", fontsize = 16)
plt.ylabel(r"Residuals", fontsize = 16) # $\frac{\delta_{ANN} -\delta_{DM}}{\delta_{DM} +1 } $
plt.legend()
plt.axis([0,20.0,-1,2.0])
plt.show()

In [17]:
plt.figure()
plt.plot(delta_DM, delta_ANN,'o', color = '#e0ffff', markersize = .5, alpha = .1) #,alpha =.55, markersize = .5
plt.plot(t,t,'--',color = '#228B22',lw=2,alpha=.7, label = r"$\delta_{ANN} = \delta_{DM}$")
plt.xlabel(r"$\delta_{DM} + 1 $", fontsize = 16)
plt.ylabel(r"$\delta_{ANN} + 1 $", fontsize = 16) #48d1cc
plt.title(r"")
plt.axis([0,3.5,0,3.5])
plt.plot(bins_True,median_True,color='red',linestyle = '-',lw=2.5,alpha=.7,label = r"Median$\left(\{\delta_i\}<\delta_c\right)$") 
plt.plot(bins_False,median_False,color='blue',linestyle = '-',lw=2.5,alpha=.7,label = r"Median$\left(\delta_i > \delta_c\right)$") 
plt.errorbar(bins_True[bin_list_errobar_T],median_True[bin_list_errobar_T], yerr = [median_True[bin_list_errobar_T]-running_err_down_T,running_err_up_T-median_True[bin_list_errobar_T]],alpha = .8, fmt = None, elinewidth = 1.5, ecolor = 'red')
plt.errorbar(bins_False[bin_list_errobar_F],median_False[bin_list_errobar_F], yerr = [median_False[bin_list_errobar_F]-running_err_down_F,running_err_up_F-median_False[bin_list_errobar_F]],alpha = .8, fmt = None, elinewidth = 1.5, ecolor = 'blue')
pylab.legend(loc='lower right',frameon=True, prop={'size':14})
plt.show()

/usr/lib/pymodules/python2.7/matplotlib/collections.py:548: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if self._edgecolors == 'face':


In [ ]:
# ================================================ SAVE DATA ===========================================================

DM_train = []
Halos_train = []

DM_test = []
Halos_test = []

Condition = []

for line in file('RANDOM_SAMPLE.txt'):
        row = line.split(" ")
        DM_train.append(float(row[0]))
        Halos_train.append([float(x) for x in row[1:]])

for line in file('test_deltas.txt'):
        row = line.split(" ")
        Condition.append(int(row[0]))
        DM_test.append(float(row[1]))
        Halos_test.append([float(x) for x in row[2:]])
    
        
DM_train_np = np.array(DM_train).astype(dtype = np.float32) 
DM_test_np = np.array(DM_test).astype(dtype = np.float32)

Halos_train_np = np.array(Halos_train).astype(dtype= np.float32) # GUARDAR
Halos_test_np = np.array(Halos_test).astype(dtype = np.float32)

Condition_np = np.array(Condition)

minval, maxval =  DM_train_np.min(), DM_train_np.max()

mask = ( DM_test_np >= minval ) & (DM_test_np <= maxval)
Halos_test_np = Halos_test_np[mask]
Condition_np = Condition_np[mask]
DM_test_np = DM_test_np[mask]

np.savetxt('TEST_DELTAS.txt', np.array([Condition_np, DM_test_np]).T, fmt = ['%i','%.5f'] )

labels_train =  ToClass(DM_train_np, NUM_LABELS, minval, maxval) # GUARDAR
labels_test  =  ToClass(DM_test_np, NUM_LABELS, minval, maxval) # GUARDAR

print minval
print maxval

del(DM_test_np,DM_test, DM_train_np)
del DM_train[:]
del Halos_train[:]




In [ ]:
temp1 = np.zeros((len(labels_train),28)) 
temp2 = np.zeros((len(labels_test),28))

for i in range(len(labels_train)):
    temp1[i] = np.concatenate(([labels_train[i]], Halos_train_np[i])) # Halos_train

for j in range(len(labels_test)):
    temp2[j] = np.concatenate(([labels_test[j]], Halos_test_np[j])) # Halos test

In [ ]:
formatt = []
formatt.append('%i')
for i in range(27):
    formatt.append('%.5f')

#np.savetxt('RANDOM_SAMPLE_BIN.txt', temp1, fmt = formatt)
np.savetxt('TESTDATA.txt', temp2, fmt = formatt)    